In [1]:
# source ~/mx_venv/bin/activate
# cd /home/macul/libraries/MXNet2Caffe
# python -m json2prototxt --mx-json /media/macul/black/mxnet_training/r50/dgx_train1/train_1-symbol.json --cf-prototxt /media/macul/black/mxnet_training/r50/dgx_train1/model-caffe-converted.prototxt
# add below layer after 'out_relu3', and set flatten as the input to next layer
'''
layer {
  bottom: "out_relu3"
  top: "flatten"
  name: "fc1_flatten"
  type: "Flatten"
  flatten_param { axis: 1}
}
'''
# remove all layers after 'out_embedding_scale'
# save modified file as dgx_train1_31-caffe.prototxt
# python -m mxnet2caffe --mx-model /media/macul/black/mxnet_training/r50/dgx_train1/train_1 --mx-epoch 31 --cf-prototxt /media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.prototxt --cf-model /media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe-converted.caffemodel

'\nlayer {\n  bottom: "bn1"\n  top: "flatten"\n  name: "fc1_flatten"\n  type: "Flatten"\n  flatten_param { axis: 1}\n}\n'

In [1]:
import sys
sys.path.append('/home/macul/libraries/MXNet2Caffe')
sys.path.append('/home/macul/insightface/src/')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxiter')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxconv')
import mxnet as mx
import caffe
from collections import namedtuple
import numpy as np
import cv2

In [2]:
#mxModelPath = '/media/macul/black/mxnet_training/r50/dgx_train1/train_1'
#mxModelEpoch = 31
mxModelPath = '/media/macul/black/mxnet_training/r50/server_train18/train_18'
mxModelEpoch = 6062

sym_tmp, arg_params, aux_params = mx.model.load_checkpoint(mxModelPath, mxModelEpoch)

In [3]:
all_layers = sym_tmp.get_internals()
output_list = all_layers.list_outputs()
sym3 = all_layers['out_embedding_output']
#sym3 = all_layers['out_fc1_output']
model = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
model.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
model.set_params(arg_params, aux_params, allow_missing=False, allow_extra=True)

In [200]:
output_list

['data',
 'stem_conv1_weight',
 'stem_conv1_output',
 'stem_bn2_gamma',
 'stem_bn2_beta',
 'stem_bn2_moving_mean',
 'stem_bn2_moving_var',
 'stem_bn2_output',
 'stem_relu1_gamma',
 'stem_relu1_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_prelu2_gamma',
 'stage1_unit1_prelu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn4_gamma',
 'stage1_unit1_bn4_beta',
 'stage1_unit1_bn4_moving_mean',
 'stage1_unit1_bn4_moving_var',
 'stage1_unit1_bn4_output',
 'stage1_unit1_convr_weight',
 'stage1_unit1_convr_output',
 'stage1_unit1_bnr_gamma',
 'stage1_unit1_bnr_beta',
 'stage1_unit1_bnr_moving_mean',
 'stage1_unit1_bnr_moving_var',
 

In [4]:
img_normalization = True
img = mx.image.imread('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png')
img = img.astype('float32')
if img_normalization:
    img = img - 127.5
    img = img * 0.0078125
img = img.transpose((2,0,1))
img = img.expand_dims(axis=0)   
Batch = namedtuple('Batch',['data'])
model.forward(Batch([img]))

In [5]:
outs=model.get_outputs()

In [6]:
out_mx=outs[0].asnumpy()[0]

In [7]:
print(arg_params['out_fc1_bias'].shape)
print(arg_params['out_fc1_weight'].shape)

(512L,)
(512L, 25088L)


In [9]:
arg_params['out_embedding_gamma']=1

In [11]:
cfModelProto='/media/macul/black/mxnet_training/r50/server_train18/server_train18-caffe.prototxt'
cfModelWeight='/media/macul/black/mxnet_training/r50/server_train18/server_train18_6062-caffe.caffemodel'
#cfModelProto='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.prototxt'
#cfModelWeight='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe-converted.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [12]:
def getOutput(img_path, output_name):
    mean_array = np.ones([3,112, 112])*127.5
    scale_value = 0.0078125
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = np.rollaxis(img,2)
    img = img - mean_array
    img = img * scale_value
    img = img[np.newaxis,...] 
    
    Net.blobs['data'].data[...] = img
    out = Net.forward()
    out = out[output_name][0]
    return out

In [13]:
Net.params['out_fc1'][0].data[...] = arg_params['out_fc1_weight'].asnumpy()
Net.params['out_fc1'][1].data[...] = arg_params['out_fc1_bias'].asnumpy()

In [14]:
Net.params['out_fc1'][0].data[...] - arg_params['out_fc1_weight'].asnumpy()
Net.params['out_fc1'][1].data[...] - arg_params['out_fc1_bias'].asnumpy()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [156]:
np.linalg.norm(Net.params['stem_conv1'][0].data[...])

0.74477017

In [105]:
Net.params

OrderedDict([('stem_conv1', <caffe._caffe.BlobVec at 0x7facc07634b0>),
             ('stem_bn2', <caffe._caffe.BlobVec at 0x7facc0763750>),
             ('stem_bn2_scale', <caffe._caffe.BlobVec at 0x7facc07637c0>),
             ('stem_relu1', <caffe._caffe.BlobVec at 0x7facc07638a0>),
             ('stage1_unit1_bn1', <caffe._caffe.BlobVec at 0x7facc0763830>),
             ('stage1_unit1_bn1_scale',
              <caffe._caffe.BlobVec at 0x7facc07636e0>),
             ('stage1_unit1_conv1', <caffe._caffe.BlobVec at 0x7facc0763910>),
             ('stage1_unit1_bn2', <caffe._caffe.BlobVec at 0x7facc0763980>),
             ('stage1_unit1_bn2_scale',
              <caffe._caffe.BlobVec at 0x7facc07639f0>),
             ('stage1_unit1_prelu2', <caffe._caffe.BlobVec at 0x7facc0763a60>),
             ('stage1_unit1_conv2', <caffe._caffe.BlobVec at 0x7facc0763ad0>),
             ('stage1_unit1_bn4', <caffe._caffe.BlobVec at 0x7facc0763b40>),
             ('stage1_unit1_bn4_scale',
           

In [157]:
print(np.sum(Net.params['out_embedding_scale'][1].data[...]-arg_params['out_embedding_beta'].asnumpy()))
print(np.sum(Net.params['out_embedding_scale'][0].data[...]-arg_params['out_embedding_gamma'].asnumpy()))
print(np.sum(Net.params['out_embedding'][0].data[...]-aux_params['out_embedding_moving_mean'].asnumpy()))
print(np.sum(Net.params['out_embedding'][1].data[...]-aux_params['out_embedding_moving_var'].asnumpy()))

0.0
0.0
0.0
0.0


In [173]:
arg_params['out_embedding_gamma'].asnumpy()

array([6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42,
       6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e-42, 6.66e

In [17]:
Net.params['out_embedding_scale'][0].data[...]=1.0

In [18]:
out_caffe = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png', 'out_embedding')

In [19]:
np.dot(out_caffe/np.linalg.norm(out_caffe),out_mx/np.linalg.norm(out_mx))

0.9999999

In [20]:
Net.save('/media/macul/black/mxnet_training/r50/server_train18/server_train18_6062-caffe.caffemodel')

In [21]:
#cfModelProto='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.prototxt'
#cfModelWeight='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.caffemodel'
cfModelProto='/media/macul/black/mxnet_training/r50/server_train18/server_train18-caffe.prototxt'
cfModelWeight='/media/macul/black/mxnet_training/r50/server_train18/server_train18_6062-caffe.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [22]:
out_caffe_new = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png', 'out_embedding')

In [23]:
np.dot(out_caffe_new/np.linalg.norm(out_caffe_new),out_mx/np.linalg.norm(out_mx))

0.9999999

In [ ]:
# dgx_train1_31-caffe.prototxt and dgx_train1_31-caffe.caffemodel are converted caffe model for use
# note that the input channel should be RGB, image should be normalized, and the output embedding should be normalized by its norm

In [37]:
np.linalg.norm(out_caffe_new)

0.17938164